In [1]:
#!pip install spacy
#print(spacy.__version__)
#!pip install -U pip setuptools wheel
!pip install spacy==2.2.1
!python -m spacy download en_core_web_lg
#!pip install pickle
#!pip uninstall spacy
#!pip install blackstone
#!pip install https://blackstone-model.s3-eu-west-1.amazonaws.com/en_blackstone_proto-0.0.1.tar.gz
#!pip install "spacy-stanza<0.3.0"

     |████████████████████████████████| 10.2 MB 5.3 MB/s 
     |████████████████████████████████| 2.1 MB 56.1 MB/s 
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 2.2.5 requires spacy>=2.2.2, but you have spacy 2.2.1 which is incompatible.
     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=802b2780802a5

In [ ]:
#stanza.download("en")

In [1]:
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.matcher import Matcher
from spacy import displacy
from google.colab import files
import os

In [2]:
print("spacy version: " + spacy.__version__)

spacy version: 2.2.1


In [ ]:
#nlp = spacy.load("es_core_web_lg")
#nlp = spacy.load("Modelo")es_core_news_md

### Generamos el modelo

In [4]:
#!/usr/bin/env python
# coding: utf8

# Training additional entity types using spaCy

# New entity labels
# Specify the new entity labels which you want to add herehttps
LABEL = ['PROFESION','DNI','DOC','CARDINAL','PERSON','GPE','TIME','ORG','LOC']

"""
Se agregaron los labels 'PROFESION', 'DNI Y 'DOC'
"""

# Loading training data 
with open ('NLP.pickle', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

#@plac.annotations(
#    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#    new_model_name=("New model name for model meta.", "option", "nm", str),
#    output_dir=("Optional output directory", "option", "o", Path),
#    n_iter=("Number of training iterations", "option", "n", int))

def new_model(model='en_core_web_lg', new_model_name='en_core_web_lg_trained', output_dir=None, n_iter=300):
    """Setting up the pipeline and entity recognizer, and training the new entity."""
    if model is not None:
        nlp = spacy.load('en_core_web_lg')  # load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe('ner')

    for i in LABEL:
        ner.add_label(i)   # Add new entity labels to entity recognizer

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.entity.create_optimizer()

    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)
            print('Losses', losses)

    # Test the trained model
    test_text='PUBLIC DEED OF SALE. NUMBER THOUSAND THREE HUNDRED FIFTEEN (1315). In Alicante, on the fourteenth of October, nineteen ninety-seven. Before me, RAFAEL MUÑOZ GUILLENA, Notary Public of the Illustrious College of Alicante, resident in Playa de San Juan, district of Alicante, APPEARS: The spouses DON ISMAEL GARCIA NAVARRO and DOÑA ISIDORA MARTINEZ MORENO, of legal age, mechanic he and manager her, neighbors of Montornès del Vallès, Castell street, number 3, with DDNNII-NNIIFF 27,322,177-M and 51,347,341-P. They are subject to the regime economic matrimonial property separation. And the spouses DON JUAN LOPEZ FERNANDEZ and DOÑA ROSA FERNANDEZ SANCHEZ, of legal age, merchant he and housewife her, neighbors of Montornés del Vallés, Barcelona street, number 16, 4º, 1st, with DDNNII-NNIIFF 31,633,237-F and 21,427,427-A. They are subject to civil law that determines your Catalan neighborhood by continuous ten-year residence and at matrimonial property regime of community property. I make sure of your identity by reviewed documentation. Personal circumstances result from your statement. They INTERVENE on their own behalf.'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = 'en_core_web_lg_trained'  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # Test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

new_model(output_dir='Modelo')

Loaded model 'en_core_web_lg'
Losses {'ner': 184.50290870666504}
Losses {'ner': 195.71167570352554}
Losses {'ner': 179.25467633455992}
Losses {'ner': 183.333707041038}
Losses {'ner': 173.6921855211258}
Losses {'ner': 145.96220195293427}
Losses {'ner': 155.9833436012268}
Losses {'ner': 160.353613615036}
Losses {'ner': 170.32531654834747}
Losses {'ner': 158.3687779903412}
Losses {'ner': 160.85402417182922}
Losses {'ner': 142.72556948661804}
Losses {'ner': 144.0771975517273}
Losses {'ner': 154.0332338809967}
Losses {'ner': 148.56422591209412}
Losses {'ner': 155.41088151931763}
Losses {'ner': 161.98370300233364}
Losses {'ner': 149.16146659851074}
Losses {'ner': 148.75285004079342}
Losses {'ner': 147.7769637107849}
Losses {'ner': 147.45576667785645}
Losses {'ner': 147.96092057228088}
Losses {'ner': 138.16510270535946}
Losses {'ner': 138.05439165234566}
Losses {'ner': 147.68709707260132}
Losses {'ner': 140.474535882473}
Losses {'ner': 143.81151580810547}
Losses {'ner': 134.06694847345352}
Lo

### Cargamos el modelo

In [5]:
#nlp = spacy.load("en_core_web_lg")
nlp = spacy.load("Modelo")
nlp.Defaults.stop_words |= {'Before me' , 'APPEARS:' , 'DON', 'DOÑA', 'In', 'and'}
print(nlp.Defaults.stop_words)

{'using', 'show', 'in', 'everyone', 'first', 'ourselves', 'many', 'your', 'just', 'other', 'from', 'them', 'herein', 'be', 'were', 'a', 'before', 'give', 'towards', 'was', 'becomes', 'thru', 'across', 'none', 'behind', 'its', 'hundred', 'used', 'something', 'such', 'call', 'our', 'whither', 'otherwise', 'ten', 'becoming', 'who', 'latter', '‘ve', 'sometimes', 'wherever', 'thereafter', 'can', 'by', 'per', 'formerly', 'and', 'himself', 'nobody', 'to', 'part', 'nothing', 'became', 'am', 'eleven', 'former', 'n‘t', 'even', 'four', "'ve", 'along', 'also', 'around', 'else', 'nor', 'In', 'within', 'eight', 'both', 'not', 're', 'when', 'hereafter', 'indeed', 'him', 'it', 'upon', 'DON', 'than', 'rather', 'been', 'same', 'seems', 'further', "'ll", 'few', 'their', 'although', 'at', 'never', '’ll', 'meanwhile', 'itself', 'she', 'therefore', 'has', 'these', 'anyhow', 'APPEARS:', 'without', '‘s', 'should', 'say', 'therein', 'beforehand', 'the', 'moreover', 'make', 'mine', 'anywhere', 'front', 'see', '

In [6]:
def extract_entities(doc, head_label):
    ent = ''
    sep=' # '
    for entity in doc.ents:

        head_label= entity.sent.root.text
        if entity.sent.root.pos_ == 'VERB':
            continue
        if ent == '':
            ent = ent + '(' + head_label + sep + entity.label_ + sep + entity.text + ')'
        else:
            ent = ent + ' ' + '(' + head_label + sep + entity.label_ + sep + entity.text + ')'

        #if entity.label_ == "PERSON": head_label = entity.text
        #if entity.n_lefts > 0:
        #   head_label = entity.lefts[0].text
        #   lefts = [t.text for t in entity.lefts if t.pos_ == 'NOUN'] 
        #   if len(lefts) > 0 :
        #      head_label = lefts[0]

    if ent!='': print(ent)
    return head_label

In [7]:
def matcher_train():
    matcher = Matcher(nlp.vocab)

    # TITULO
    pattern = [{"IS_UPPER": True, "TEXT": {"REGEX": ".+"}}]
    matcher.add("Escritura_TITULO", [pattern])

    # NUMERO
    pattern = [{"TEXT": {"REGEX": "[^(NUMBER)\b].+"}}]
    matcher.add("Escritura_NUMERO", [pattern])
    
    return matcher

In [8]:
def extract_matcher(doc, matcher):
    matches = matcher(doc)
    print(matches)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)

### Cargamos texto de prueba

In [9]:
#text='RAFAEL MUÑOZ GUILLENA, Notary Public of the Illustrious College of Alicante, from Playa de San Juan, district of Alicante. ISMAEL GARCIA NAVARRO, mechanic, with DDNNII-NNIIFF 27,322,177, neighbor of Montornès del Vallès, Castell street, number 3. ISIDORA MARTINEZ MORENO, administrator, with DDNNII-NNIIFF 51,347,341, of legal age, neighbor of Montornès del Vallès, Castell street, number 3.'
text='PUBLIC DEED OF SALE. NUMBER THOUSAND THREE HUNDRED FIFTEEN (1315). In Alicante, on the fourteenth of October, nineteen ninety-seven. Before me, RAFAEL MUÑOZ GUILLENA, Notary Public of the Illustrious College of Alicante, resident in Playa de San Juan, district of Alicante. The spouses MANUEL LEMA MULLO and ISIDORA MARTINEZ MORENO, of legal age, mechanic he and manager her, from Montornès del Vallès, Castell street, number 3, DDNNII-NNIIFF 27,322,177-M, 51,347,341-P. They are subject to the regime economic matrimonial property separation. The spouses JUAN LOPEZ FERNANDEZ and ROSA FERNANDEZ SANCHEZ, of legal age, merchant he and housewife her, from Montornés del Vallés, Barcelona street, number 16, 4º, 1st, with DDNNII-NNIIFF 31,633,237-F, 21,427,427-A. They are subject to civil law that determines your Catalan neighborhood by continuous ten-year residence and at matrimonial property regime of community property. I make sure of your identity by reviewed documentation. Personal circumstances result from your statement. They INTERVENE on their own behalf.'
sentences = text.split(".")
#matcher = matcher_train()

In [10]:
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in nlp.Defaults.stop_words]
    return (" ").join(tokens_filtered)

In [11]:
head_label = 'Escritura'
for sentence in sentences:
    sentence = remove_mystopwords(sentence)
    doc = nlp(sentence)
    #print(doc.ents)
    head_label = extract_entities(doc, head_label)
    #for token in doc:
    #    print(token.text, token.pos_ ,token.tag_, token.dep_)


(DEED # DOC # PUBLIC DEED OF SALE)
(FIFTEEN # CARDINAL # THOUSAND THREE HUNDRED FIFTEEN) (FIFTEEN # CARDINAL # 1315)
(Alicante # GPE # Alicante) (Alicante # TIME # fourteenth October) (Alicante # TIME # nineteen ninety-seven)
(GUILLENA # PERSON # RAFAEL MUÑOZ GUILLENA) (GUILLENA # PROFESION # Notary Public) (GUILLENA # ORG # Illustrious College Alicante) (GUILLENA # LOC # Playa de San Juan) (GUILLENA # GPE # Alicante)
(spouses # PERSON # MANUEL LEMA MULLO) (spouses # PERSON # ISIDORA MARTINEZ MORENO) (spouses # PROFESION # mechanic) (spouses # PROFESION # manager) (spouses # LOC # Montornès del Vallès) (spouses # LOC # Castell street, number 3) (spouses # DOC # DDNNII-NNIIFF) (spouses # DNI # 27,322,177-M) (spouses # DNI # 51,347,341-P)
(spouses # PERSON # JUAN LOPEZ FERNANDEZ) (spouses # PERSON # ROSA FERNANDEZ SANCHEZ) (spouses # PROFESION # merchant) (spouses # PROFESION # housewife) (spouses # LOC # Montornés del Vallés) (spouses # LOC # Barcelona street, number 16, 4º, 1st) (spous

In [12]:
doc = nlp(text)

In [ ]:
for token in doc:
    print(token.text, token.pos_ ,token.tag_, token.dep_)

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

PUBLIC public ADJ JJ amod XXXX True False
DEED deed NOUN NN ROOT XXXX True False
OF of ADP IN prep XX True True
SALE sale NOUN NN pobj XXXX True False
. . PUNCT . punct . False False
NUMBER number NOUN NN compound XXXX True False
THOUSAND thousand NUM CD compound XXXX True False
THREE three NUM CD compound XXXX True True
HUNDRED hundred NUM CD compound XXXX True True
FIFTEEN fifteen NUM CD ROOT XXXX True True
( ( PUNCT -LRB- punct ( False False
1315 1315 NUM CD appos dddd False False
) ) PUNCT -RRB- punct ) False False
. . PUNCT . punct . False False
In in ADP IN prep Xx True True
Alicante Alicante PROPN NNP pobj Xxxxx True False
, , PUNCT , punct , False False
on on ADP IN prep xx True True
the the DET DT det xxx True True
fourteenth fourteenth NOUN NN pobj xxxx True False
of of ADP IN prep xx True True
October October PROPN NNP pobj Xxxxx True False
, , PUNCT , punct , False False
nineteen nineteen NUM CD nummod xxxx True False
ninety ninety NUM CD compound xxxx True False
- - PUNCT 

In [13]:
displacy.render(doc, jupyter=True ,style="dep")

## Stanford Model

In [ ]:

import stanza
from spacy_stanza import StanzaLanguage

snlp = stanza.Pipeline(lang="en")
nlp = StanzaLanguage(snlp)

2021-11-25 20:29:43 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-11-25 20:29:44 INFO: Use device: cpu
2021-11-25 20:29:44 INFO: Loading: tokenize
2021-11-25 20:29:44 INFO: Loading: pos
2021-11-25 20:29:45 INFO: Loading: lemma
2021-11-25 20:29:45 INFO: Loading: depparse
2021-11-25 20:29:46 INFO: Loading: sentiment
2021-11-25 20:29:46 INFO: Loading: ner
2021-11-25 20:29:47 INFO: Done loading processors!


In [ ]:
#!/usr/bin/env python
# coding: utf8

# Training additional entity types using spaCy

# New entity labels
# Specify the new entity labels which you want to add here
LABEL = ['PROFESION','DNI','DOC','CARDINAL','PERSON','GPE','TIME','ORG','LOC']

"""
Se agregaron los labels 'PROFESION', 'DNI Y 'DOC'
"""

# Loading training data 
with open ('NLP.pickle', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

#@plac.annotations(
#    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#    new_model_name=("New model name for model meta.", "option", "nm", str),
#    output_dir=("Optional output directory", "option", "o", Path),
#    n_iter=("Number of training iterations", "option", "n", int))

def new_model(model='/root/stanza_resourcers/en', new_model_name='en_blackstone_proto_trained', output_dir=None, n_iter=150):
    """Setting up the pipeline and entity recognizer, and training the new entity."""
    if model is not None:
        nlp = spacy.load('en_blackstone_proto')  # load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe('ner')

    for i in LABEL:
        ner.add_label(i)   # Add new entity labels to entity recognizer

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.entity.create_optimizer()

    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)
            print('Losses', losses)

    # Test the trained model
    test_text='PUBLIC DEED OF SALE. NUMBER THOUSAND THREE HUNDRED FIFTEEN (1315). In Alicante, on the fourteenth of October, nineteen ninety-seven. Before me, RAFAEL MUÑOZ GUILLENA, Notary Public of the Illustrious College of Alicante, resident in Playa de San Juan, district of Alicante, APPEARS: The spouses DON ISMAEL GARCIA NAVARRO and DOÑA ISIDORA MARTINEZ MORENO, of legal age, mechanic he and administrative her, neighbors of Montornès del Vallès, Castell street, number 3, with DDNNII-NNIIFF 27,322,177-M and 51,347,341 P, respectively.'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = 'en_blackstone_proto_trained'  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # Test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

new_model(output_dir='Modelo')

Loaded model '/root/stanza_resourcers/en'
Losses {'ner': 84.83442878723145}
Losses {'ner': 86.51740264892578}
Losses {'ner': 84.7733383178711}
Losses {'ner': 79.63180637359619}
Losses {'ner': 81.17196846008301}
Losses {'ner': 76.35544490814209}
Losses {'ner': 82.2359390258789}
Losses {'ner': 76.30554866790771}
Losses {'ner': 70.94417762756348}
Losses {'ner': 63.66588115692139}
Losses {'ner': 74.5754280090332}
Losses {'ner': 64.63926410675049}
Losses {'ner': 71.92570114135742}
Losses {'ner': 65.38386726379395}
Losses {'ner': 78.3033676147461}
Losses {'ner': 65.79044723510742}
Losses {'ner': 73.33080959320068}
Losses {'ner': 73.52745628356934}
Losses {'ner': 70.90248203277588}
Losses {'ner': 68.96009635925293}
Losses {'ner': 72.67839908599854}
Losses {'ner': 65.69625282287598}
Losses {'ner': 71.77482795715332}
Losses {'ner': 63.66556453704834}
Losses {'ner': 77.45801734924316}
Losses {'ner': 74.1164608001709}
Losses {'ner': 74.12934494018555}
Losses {'ner': 73.1406774520874}
Losses {'ner

In [ ]:
head_label = 'Escritura'
for sentence in sentences:
    doc = nlp(sentence)
    head_label = extract_entities(doc, head_label)

(DEED # DOC # PUBLIC DEED OF SALE)
(FIFTEEN # CARDINAL # THOUSAND THREE HUNDRED FIFTEEN) (FIFTEEN # CARDINAL # 1315)
(seven # GPE # Alicante) (seven # TIME # fourteenth of October) (seven # TIME # nineteen ninety-seven)
(GUILLENA # PERSON # RAFAEL MUÑOZ GUILLENA) (GUILLENA # PROFESION # Notary Public) (GUILLENA # ORG # Illustrious College of Alicante) (GUILLENA # LOC # Playa de San Juan) (GUILLENA # GPE # Alicante)
(spouses # PERSON # MANUEL LEMA MULLO) (spouses # PERSON # ISIDORA MARTINEZ MORENO) (spouses # PROFESION # mechanic) (spouses # PROFESION # manager) (spouses # LOC # Montornès del Vallès) (spouses # LOC # Castell street, number 3) (spouses # DOC # DDNNII-NNIIFF) (spouses # DNI # 27,322,177-M) (spouses # DNI # 51,347,341-P)
(spouses # PERSON # JUAN LOPEZ FERNANDEZ) (spouses # PERSON # ROSA FERNANDEZ SANCHEZ) (spouses # PROFESION # merchant) (spouses # PROFESION # housewife) (spouses # LOC # Montornés del Vallés) (spouses # LOC # Barcelona street, number 16, 4º, 1st) (spouses 

In [ ]:
!zip -r Modelo.zip Modelo

  adding: Modelo/ (stored 0%)
  adding: Modelo/meta.json (deflated 46%)
  adding: Modelo/tokenizer (deflated 79%)
  adding: Modelo/vocab/ (stored 0%)
  adding: Modelo/vocab/lookups.bin (deflated 45%)
  adding: Modelo/vocab/key2row (deflated 13%)
  adding: Modelo/vocab/lexemes.bin (deflated 64%)
  adding: Modelo/vocab/strings.json (deflated 61%)
  adding: Modelo/vocab/vectors (deflated 8%)
  adding: Modelo/parser/ (stored 0%)
  adding: Modelo/parser/cfg (deflated 46%)
  adding: Modelo/parser/moves (deflated 56%)
  adding: Modelo/parser/model (deflated 7%)
  adding: Modelo/tagger/ (stored 0%)
  adding: Modelo/tagger/cfg (deflated 34%)
  adding: Modelo/tagger/tag_map (deflated 50%)
  adding: Modelo/tagger/model (deflated 7%)
  adding: Modelo/ner/ (stored 0%)
  adding: Modelo/ner/cfg (deflated 47%)
  adding: Modelo/ner/moves (deflated 78%)
  adding: Modelo/ner/model (deflated 6%)


In [ ]:
!zip "Modelo2.zip" "Modelo"

In [ ]:
displacy.render(doc, jupyter=True ,style="dep")